## Imports

In [14]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import os
import shutil

## Start Spark Session

In [15]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura de Arquivo de Formato Fixo") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [16]:
# Define caminho local onde será armazenada a tabela Delta
delta_path_cotahist = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cotahist"
delta_path_cotacao_historica = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica"    

## Create table bronze

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cotahist):
    shutil.rmtree(delta_path_cotahist)
    print(f"✅ Diretório removido: {delta_path_cotahist}")

## Create table cotahist
spark.sql(f"""
CREATE TABLE IF NOT EXISTS raw_cotahist (
  tipo_registro STRING,
  data_pregao STRING,
  codigo_bdi STRING,
  codigo_negociacao STRING,
  tipo_mercado STRING,
  nome_resumido_empresa STRING,
  especificacao_papel STRING,
  prazo_dias_mercado STRING,
  moeda_referencia STRING,
  preco_abertura_papel STRING,
  preco_maximo_papel STRING,
  preco_minimo_papel STRING,
  preco_medio_papel STRING,
  preco_ultimo_negocio STRING,
  preco_melhor_oferta_compra STRING,
  preco_melhor_oferta_venda STRING,
  numero_negocios_efetuados STRING,
  quantidade_total_titulos STRING,
  volume_total_titulos STRING,
  preco_exercicio_opcoes STRING,
  indicador_correcao_precos STRING,
  data_vencimento_opcoes STRING,
  fator_cotacao_papel STRING,
  preco_exercicio_pontos STRING,
  codigo_papel_sistema STRING,
  numero_distribuicao_papel STRING
)
USING DELTA
LOCATION '{delta_path_cotahist}'
""")

print("✅ Tabela raw_cotahist criada com sucesso!")

DataFrame[]

## Create table silver

In [17]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cotacao_historica):
    shutil.rmtree(delta_path_cotacao_historica)
    print(f"✅ Diretório removido: {delta_path_cotacao_historica}")

## Create table stg_cotacao_historica
spark.sql(f"""
CREATE TABLE IF NOT EXISTS stg_cotacao_historica (
  tp_registro STRING COMMENT 'Tipo do registro no arquivo',
  dt_pregao DATE COMMENT 'Data do pregão',
  cd_bdi STRING COMMENT 'Código BDI (identificação da ação)',
  cd_negociacao STRING COMMENT 'Código de negociação do papel',
  tp_mercado STRING COMMENT 'Tipo de mercado',
  nm_empresa STRING COMMENT 'Nome resumido da empresa',
  ds_especificacao_papel STRING COMMENT 'Especificação do papel',
  nr_prazo_dias_mercado STRING COMMENT 'Prazo em dias de mercado',
  cd_moeda_referencia STRING COMMENT 'Moeda de referência',
  vl_abertura DECIMAL(11,2) COMMENT 'Preço de abertura do papel',
  vl_maximo DECIMAL(11,2) COMMENT 'Preço máximo do papel',
  vl_minimo DECIMAL(11,2) COMMENT 'Preço mínimo do papel',
  vl_medio DECIMAL(11,2) COMMENT 'Preço médio do papel',
  vl_ultimo_negocio DECIMAL(11,2) COMMENT 'Preço do último negócio',
  vl_melhor_oferta_compra DECIMAL(11,2) COMMENT 'Preço da melhor oferta de compra',
  vl_melhor_oferta_venda DECIMAL(11,2) COMMENT 'Preço da melhor oferta de venda',
  qt_negocios INT COMMENT 'Número de negócios efetuados',
  qt_total_titulos INT COMMENT 'Quantidade total de títulos negociados',
  vl_total_volume DECIMAL(16,2) COMMENT 'Volume total negociado',
  vl_exercicio_opcoes DECIMAL(11,2) COMMENT 'Preço de exercício de opções',
  cd_indicador_correcao STRING COMMENT 'Indicador de correção de preços',
  dt_vencimento_opcoes DATE COMMENT 'Data de vencimento das opções',
  cd_fator_cotacao STRING COMMENT 'Fator de cotação do papel',
  vl_exercicio_pontos DECIMAL(7,6) COMMENT 'Preço de exercício em pontos',
  cd_papel_sistema STRING COMMENT 'Código do papel no sistema',
  nr_distribuicao_papel STRING COMMENT 'Número de distribuição do papel',
  ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro'
)
USING DELTA
COMMENT 'Histórico de cotações de papéis da bolsa de valores'
LOCATION '{delta_path_cotacao_historica}'
""")

print("✅ Tabela stg_cotacao_historica criada com sucesso!")

✅ Diretório removido: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
✅ Tabela stg_cotacao_historica criada com sucesso!
✅ Tabela stg_cotacao_historica criada com sucesso!


## Stop Spark Session

In [18]:
# Encerra a SparkSession
spark.stop()